In [ ]:
just_checking_integrity=False

In [ ]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
URM_test=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_test_final.npz")

In [ ]:
URM_test.shape

(4439967, 23690)

In [ ]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [ ]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [ ]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [ ]:
is_sorted(Session_ids)

True

In [ ]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["Graph"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/final/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading Graph
8321415
loaded Graph


In [ ]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids


In [ ]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [ ]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [ ]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/final/candidates.csv")
# submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [ ]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/final/candidates.csv")
# submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [ ]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/final/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [ ]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# del submission_df
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
# del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [ ]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [ ]:
dataset.fillna(0,inplace=True)

In [ ]:
gc.collect()

2552

In [ ]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN_final.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [ ]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [ ]:
dataset.head()

,Item_ID,Score_Graph,Session_Id,score_LSTM,rank_LSTM,Max_Score_Graph,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.006554,61,0.001511,45.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,23,0.007389,61,0.000000,0.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,25,0.004554,61,0.000000,0.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,69,0.011284,61,0.002591,25.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,78,0.006321,61,0.001320,49.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
gc.collect()

0

In [ ]:
dataset=dataset.sort_values("Session_Id")

In [ ]:
gc.collect()

0

In [ ]:
k=10
result = []

In [ ]:
import xgboost as xgb

result = []

for index in range(k):
    model = xgb.Booster({'predictor': 'cpu_predictor'})  # Initializing a new XGBoost model
    model.load_model(f'./drive/MyDrive/Recommendation_system/rankers/xgboost/graph_and_lstm/model_{index}.xgb')
    result.append(model.predict(xgb.DMatrix(dataset.drop(["Session_Id", "Item_ID"], axis=1))))
    # dataset[f"score_{index}"] = model.predict(xgb.DMatrix(dataset.drop(["Session_Id", "Item_ID"], axis=1)))

# combine scores
result = np.array(result)
result

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:50:42] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:50:47] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:50:52] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:50:58] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:51:02] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn

array([[-0.69083446, -0.80879563, -0.8201371 , ..., -0.30958974,
        -0.35264802, -0.7370936 ],
       [-0.6555031 , -0.8017512 , -0.8387647 , ..., -0.31247655,
        -0.32804647, -0.61308694],
       [-0.74490166, -0.84401625, -0.87903404, ..., -0.34839535,
        -0.41360682, -0.7571262 ],
       ...,
       [-0.83810234, -1.0162022 , -1.0517013 , ..., -0.41326147,
        -0.49109676, -0.8678312 ],
       [-0.9302469 , -1.112003  , -1.144232  , ..., -0.43842307,
        -0.54761165, -0.9003043 ],
       [-1.3186029 , -1.527393  , -1.5930198 , ..., -0.7111834 ,
        -0.7645064 , -1.135615  ]], dtype=float32)

In [ ]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

In [ ]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

14

In [ ]:
dataset.head()

,Session_Id,Item_ID,score
0,61,11,-0.794034
127,61,3806,-0.936480
128,61,3815,-0.963776
129,61,3826,-0.963776
130,61,3844,-0.818024


In [ ]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [ ]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [ ]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [ ]:
dataset.iloc[0]

Session_Id                                                   61
Item_ID       [11, 3806, 3815, 3826, 3844, 3904, 4071, 4083,...
score         [-0.7940338253974915, -0.936479926109314, -0.9...
indices       [106, 137, 45, 139, 165, 20, 5, 37, 30, 44, 13...
Name: 0, dtype: object

In [ ]:
dataset.iloc[0]["Item_ID"][126]

759

In [ ]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [ ]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [ ]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [ ]:
dataset

,Session_Id,predictions,rank
0,61,"[1045, 292, 3586, 429, 1563, 4574, 3904, 3414,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[4148, 1183, 3564, 4758, 3778, 3705, 2346, 168...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4148, 4146, 1936, 2330, 2704, 3326, 3827, 134...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[4202, 3338, 16, 2769, 4990, 2448, 2180, 4989,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 2970, 3441, 3555, 2471, 2899, 1070, 473...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 1468, 4026, 772, 3656, 191, 3424, 3037, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 3223, 4586, 3550, 3880, 3505, 242...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[4865, 819, 603, 4966, 2724, 779, 4418, 3879, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 1972, 2285, 4220, 292, 3414, 4740, 3586,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [ ]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [ ]:
dataset.columns=["session_id","code","rank"]

In [ ]:
dataset.head()

,session_id,code,rank
0,61,"(1045, 292, 3586, 429, 1563, 4574, 3904, 3414,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(4148, 1183, 3564, 4758, 3778, 3705, 2346, 168...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4148, 4146, 1936, 2330, 2704, 3326, 3827, 134...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(4202, 3338, 16, 2769, 4990, 2448, 2180, 4989,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 2970, 3441, 3555, 2471, 2899, 1070, 473...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [ ]:
mapping= pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [ ]:
print(len(submission_df))

5000000


In [ ]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [ ]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [ ]:
submission_df.to_csv(
    f'./drive/MyDrive/Recommendation_system/rankers/xgboost/graph_and_lstm/final/final_sub.csv',
    index=False
    )